In [1]:
import ultralytics
import cv2
import pandas as pd
import numpy as np
ultralytics.checks()

Ultralytics YOLOv8.0.96  Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 123.8/237.3 GB disk)


In [2]:
from ultralytics import YOLO
model = YOLO('runs/detect/train/weights/best.pt')

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [ ]:
model.train(data='web-min-project-akhir.v5i.yolov8\data.yaml',
            epochs=100,batch=5,imgsz=640,patience=5,optimizer='Adam',device=0,pretrained=True)

In [ ]:
model.val()

In [3]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + 0.25*(w-x))
            cy = (y + 0.75*(h-y))

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [ ]:
import cv2
import numpy as np
from IPython.display import Image
path = 'Desktop Screenshot 2023.06.13 - 16.03.35.51-1.png'
img = cv2.imread(path)
prevX, prevY, reX, reY = -1, -1, -1, -1
def RGB(event, x, y, flags, param):
    global prevX, prevY, reX, reY
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x,y),15,(255,255,255),-1)
        strXY = '(' + str(x) +','+str(y) + ')'
        font = cv2.FONT_HERSHEY_PLAIN
        colorsBGR = [x,y]
        cv2.putText(img, strXY, (x+10, y-10), font,1,(255,255,255))
        if prevX == -1 and prevY == -1:
            prevX, prevY = x, y
        else:
            cv2.line(img,(prevX, prevY), (x,y), (0,0,255), 5)
            prevX, prevY = x, y
            reX, reY = -1, -1
        cv2.imshow("image", img)
        print(colorsBGR)

cv2.imshow("image", img)
cv2.setMouseCallback("image", RGB)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
area=[(855, 464),(1203, 403),(1220, 418),(873, 482)]

In [5]:
class_list=['car','crosswalk','motorcycle']

In [6]:
tracker=Tracker()

In [7]:
area_c=set()

In [8]:
from IPython.display import Image

In [9]:
cap=cv2.VideoCapture('SM ISTANA PLAZA MEDAN 30FPS-1.mp4')
count=0

In [10]:
while True:
    ret,frame=cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    result=model.predict(frame)
    a=result[0].boxes.boxes.cpu().numpy()
    px = pd.DataFrame(a).astype('float')
    list_=[]
    for index,row in px.iterrows():
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if d==0 or d==2:
            list_.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list_)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+0.75*(x4-x3))
        cy=int(y3+0.75*(y4-y3))
        results=cv2.pointPolygonTest(np.array(area,np.int32),((cx,cy)),False)
        if results >=0:
            cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
            cv2.rectangle(frame,(x3,y3),(x4,y4),(255,0,0),2)
            cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
            area_c.add(id)
    cv2.polylines(frame,[np.array(area,np.int32)],True,(0,0,255),3)
    d=(len(area_c))
    cv2.putText(frame,('Pelanggaran:-')+str(d),(60,90),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.imshow("RGB",frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 384x640 3 cars, 2 crosswalks, 4 motorcycles, 39.5ms
Speed: 3.0ms preprocess, 39.5ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 4 cars, 2 crosswalks, 4 motorcycles, 16.0ms
Speed: 3.0ms preprocess, 16.0ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 4 cars, 2 crosswalks, 4 motorcycles, 14.9ms
Speed: 2.1ms preprocess, 14.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 4 cars, 2 crosswalks, 3 motorcycles, 15.7ms
Speed: 2.5ms preprocess, 15.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 4 cars, 2 crosswalks, 3 motorcycles, 15.1ms
Speed: 1.0ms preprocess, 15.1ms inference, 2.0ms postprocess per image at sh


0: 384x640 8 cars, 1 crosswalk, 11 motorcycles, 14.5ms
Speed: 2.0ms preprocess, 14.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 1 crosswalk, 11 motorcycles, 14.1ms
Speed: 3.0ms preprocess, 14.1ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 9 cars, 1 crosswalk, 9 motorcycles, 15.5ms
Speed: 2.0ms preprocess, 15.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 9 cars, 1 crosswalk, 10 motorcycles, 14.4ms
Speed: 2.0ms preprocess, 14.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 9 cars, 1 crosswalk, 14 motorcycles, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 1.5ms postprocess per image at sha

Speed: 1.0ms preprocess, 19.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 11 cars, 1 crosswalk, 13 motorcycles, 18.4ms
Speed: 2.0ms preprocess, 18.4ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 14 cars, 1 crosswalk, 12 motorcycles, 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 13 cars, 1 crosswalk, 9 motorcycles, 17.4ms
Speed: 1.0ms preprocess, 17.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 13 cars, 1 crosswalk, 9 motorcycles, 16.0ms
Speed: 3.3ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is depreca


0: 384x640 14 cars, 1 crosswalk, 10 motorcycles, 15.5ms
Speed: 2.5ms preprocess, 15.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 15 cars, 1 crosswalk, 10 motorcycles, 16.6ms
Speed: 1.0ms preprocess, 16.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 15 cars, 1 crosswalk, 11 motorcycles, 16.9ms
Speed: 1.0ms preprocess, 16.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 15 cars, 1 crosswalk, 9 motorcycles, 16.6ms
Speed: 2.0ms preprocess, 16.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 16 cars, 1 crosswalk, 8 motorcycles, 14.0ms
Speed: 2.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at

Speed: 1.0ms preprocess, 16.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 16 cars, 11 motorcycles, 14.5ms
Speed: 3.0ms preprocess, 14.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 17 cars, 7 motorcycles, 13.5ms
Speed: 2.0ms preprocess, 13.5ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 15 cars, 8 motorcycles, 15.5ms
Speed: 2.0ms preprocess, 15.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 13 cars, 7 motorcycles, 14.6ms
Speed: 2.6ms preprocess, 14.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 12 cars, 8

...

Speed: 3.2ms preprocess, 13.6ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 7 cars, 8 motorcycles, 14.5ms
Speed: 1.0ms preprocess, 14.5ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 8 motorcycles, 16.2ms
Speed: 2.0ms preprocess, 16.2ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 7 cars, 7 motorcycles, 16.5ms
Speed: 2.2ms preprocess, 16.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 5 cars, 7 motorcycles, 14.1ms
Speed: 2.0ms preprocess, 14.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 6 cars, 7 motor

WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 9 cars, 7 motorcycles, 16.5ms
Speed: 2.0ms preprocess, 16.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 6 motorcycles, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 8 motorcycles, 16.1ms
Speed: 2.0ms preprocess, 16.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 8 motorcycles, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 9 cars, 7 motorcycles, 16.5ms
Speed: 1.5ms preprocess, 16.5ms inference, 2.0ms postprocess per image at shape (1

Speed: 1.6ms preprocess, 17.0ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 8 motorcycles, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 9 cars, 8 motorcycles, 16.1ms
Speed: 2.3ms preprocess, 16.1ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 11 motorcycles, 16.4ms
Speed: 2.0ms preprocess, 16.4ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 8 motorcycles, 17.1ms
Speed: 2.0ms preprocess, 17.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 6 cars, 7 moto

WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 10 cars, 10 motorcycles, 15.1ms
Speed: 2.5ms preprocess, 15.1ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 10 cars, 9 motorcycles, 16.2ms
Speed: 1.0ms preprocess, 16.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 9 motorcycles, 15.9ms
Speed: 2.5ms preprocess, 15.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 9 motorcycles, 13.5ms
Speed: 2.0ms preprocess, 13.5ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 7 cars, 10 motorcycles, 16.1ms
Speed: 2.0ms preprocess, 16.1ms inference, 2.0ms postprocess per image at shap

WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 5 motorcycles, 18.0ms
Speed: 1.0ms preprocess, 18.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 8 motorcycles, 15.0ms
Speed: 3.0ms preprocess, 15.0ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 10 cars, 7 motorcycles, 15.5ms
Speed: 1.5ms preprocess, 15.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 8 cars, 6 motorcycles, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
WARNING  'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.

0: 384x640 10 cars, 8 motorcycles, 13.9ms
Speed: 1.5ms preprocess, 13.9ms inference, 2.0ms postprocess per image at shape 